### Problema

Em uma trasnportadora com caminhoes de espaço interno do baú de 3m³ temos que alocar produtos que em sua soma total ocupam 4.79m³, qual melhor alocação de produtos para gerar o maior lucro, tendo em mente ocupar os caminhão com os produtos com maior lucro?

In [89]:
from random import random

In [90]:
class Produto():
    def __init__(self, nome, espaco, valor):
        self.nome = nome
        self.espaco = espaco
        self.valor = valor       

In [91]:
lista_produtos = []
lista_produtos.append(Produto('Geladeira Dako', 0.751, 999.90))
lista_produtos.append(Produto('Iphone 6', 0.0000899, 2911.12))
lista_produtos.append(Produto('TV 55', 0.400, 4346.99))
lista_produtos.append(Produto('TV 50', 0.290, 3999.90))
lista_produtos.append(Produto('TV 42', 0.200, 2999.90))
lista_produtos.append(Produto('Notebook Dell', 0.00350, 2499.90))
lista_produtos.append(Produto('Ventilador Panasonic', 0.496, 199.90))
lista_produtos.append(Produto('Microondas Eletrolux', 0.0424, 308.66))
lista_produtos.append(Produto('Microondas LG', 0.0544, 429.90))
lista_produtos.append(Produto('Microondas Panasonic', 0.0319, 299.29))
lista_produtos.append(Produto('Geladeira Brastemp', 0.635, 849.00))
lista_produtos.append(Produto('Geladeira Consul', 0.870, 1199.89))
lista_produtos.append(Produto('Notebook Lenovo', 0.498, 1999.90))
lista_produtos.append(Produto('Notebook Asus', 0.527, 3999.90))

In [92]:
for produto in lista_produtos:
    print(produto)

In [93]:
class Individuo():
    def __init__(self, espacos, valores, limite_esoacos, geracao=0):
        self.espacos = espacos
        self.valores = valores
        self.limite_espacos = limite_esoacos
        self.geracao = geracao
        self.nota_avaliacao = 0
        self.espaco_usado = 0
        self.cromossomo = []

        for i in range(len(espacos)):
            if random() < 0.5:
                self.cromossomo.append('0')
            else:
                self.cromossomo.append('1')
    
    def avaliacao(self):
        nota = 0 
        soma_espacos = 0
        for i in range(len(self.cromossomo)):
            if self.cromossomo[i] == '1':
                nota += self.valores[i]
                soma_espacos += self.espacos[i]
        
        if soma_espacos > self.limite_espacos:
            nota = 1

        self.nota_avaliacao = nota
        self.espaco_usado = soma_espacos
    
    def crossover(self, outro_individuo):
        corte = round(random() * len(self.cromossomo))
        
        filho1 = outro_individuo.cromossomo[0:corte] + self.cromossomo[corte::]
        filho2 = self.cromossomo[0:corte] + outro_individuo.cromossomo[corte::]

        filhos = [Individuo(self.espacos, self.valores, self.limite_espacos, self.geracao + 1),
                  Individuo(self.espacos, self.valores, self.limite_espacos, self.geracao + 1)]

        filhos[0].cromossomo = filho1
        filhos[1].cromossomo = filho2
        
        return filhos

    def mutacao(self, taxa_mutacao):
        print('Antes %s' % self.cromossomo)
        for i in range(len(self.cromossomo)):
            if random() < taxa_mutacao:
                if self.cromossomo[i] =='1':
                    self.cromossomo[i] = '0'
                else:
                    self.cromossomo[i] = '1'
        
        print('Depois %s' % self.cromossomo)
        return self


In [94]:
espacos = []
valores = []
nomes = []
for produto in lista_produtos:
    espacos.append(produto.espaco)
    valores.append(produto.valor)
    nomes.append(produto.nome)

In [95]:
print(espacos), print(valores), print(nomes)

[0.751, 8.99e-05, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527]
[999.9, 2911.12, 4346.99, 3999.9, 2999.9, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199.89, 1999.9, 3999.9]
['Geladeira Dako', 'Iphone 6', 'TV 55', 'TV 50', 'TV 42', 'Notebook Dell', 'Ventilador Panasonic', 'Microondas Eletrolux', 'Microondas LG', 'Microondas Panasonic', 'Geladeira Brastemp', 'Geladeira Consul', 'Notebook Lenovo', 'Notebook Asus']


(None, None, None)

In [96]:
limite = 3

individuo1 = Individuo(espacos, valores, limite)

In [97]:
print('Espaços = %s' % str(individuo1.espacos))
print('Valores = %s' % str(individuo1.valores))
print('Cromossomo = %s' % str(individuo1.cromossomo))

Espaços = [0.751, 8.99e-05, 0.4, 0.29, 0.2, 0.0035, 0.496, 0.0424, 0.0544, 0.0319, 0.635, 0.87, 0.498, 0.527]
Valores = [999.9, 2911.12, 4346.99, 3999.9, 2999.9, 2499.9, 199.9, 308.66, 429.9, 299.29, 849.0, 1199.89, 1999.9, 3999.9]
Cromossomo = ['0', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1']


In [98]:
print('Componentes da carga')
for i in range(len(lista_produtos)):
    if individuo1.cromossomo[i] == '1':
        print('Nome: %s R$ %s ' % (lista_produtos[i].nome, lista_produtos[i].valor))

Componentes da carga
Nome: TV 55 R$ 4346.99 
Nome: TV 42 R$ 2999.9 
Nome: Microondas Eletrolux R$ 308.66 
Nome: Microondas LG R$ 429.9 
Nome: Microondas Panasonic R$ 299.29 
Nome: Geladeira Consul R$ 1199.89 
Nome: Notebook Lenovo R$ 1999.9 
Nome: Notebook Asus R$ 3999.9 


In [99]:
individuo1.avaliacao()

In [100]:
print('Nota: %s ' % (individuo1.nota_avaliacao))
print('Espaço usado: %s ' % (individuo1.espaco_usado))

Nota: 15584.429999999998 
Espaço usado: 2.6237000000000004 


In [101]:
print('\nIndividuo 1')
print('Componentes da carga')
for i in range(len(lista_produtos)):
    if individuo1.cromossomo[i] == '1':
        print('Nome: %s R$ %s ' % (lista_produtos[i].nome, lista_produtos[i].valor))
individuo1.avaliacao()
print('Nota: %s ' % (individuo1.nota_avaliacao))
print('Espaço usado: %s ' % (individuo1.espaco_usado))


Individuo 1
Componentes da carga
Nome: TV 55 R$ 4346.99 
Nome: TV 42 R$ 2999.9 
Nome: Microondas Eletrolux R$ 308.66 
Nome: Microondas LG R$ 429.9 
Nome: Microondas Panasonic R$ 299.29 
Nome: Geladeira Consul R$ 1199.89 
Nome: Notebook Lenovo R$ 1999.9 
Nome: Notebook Asus R$ 3999.9 
Nota: 15584.429999999998 
Espaço usado: 2.6237000000000004 


In [102]:
individuo2 = Individuo(espacos, valores, limite)

In [103]:
individuo2.avaliacao()

In [104]:
print('\nIndividuo 2')
print('Componentes da carga')
for i in range(len(lista_produtos)):
    if individuo2.cromossomo[i] == '1':
        print('Nome: %s R$ %s ' % (lista_produtos[i].nome, lista_produtos[i].valor))
individuo1.avaliacao()
print('Nota: %s ' % (individuo2.nota_avaliacao))
print('Espaço usado: %s ' % (individuo2.espaco_usado))


Individuo 2
Componentes da carga
Nome: Geladeira Dako R$ 999.9 
Nome: TV 50 R$ 3999.9 
Nome: TV 42 R$ 2999.9 
Nome: Ventilador Panasonic R$ 199.9 
Nome: Microondas LG R$ 429.9 
Nome: Notebook Lenovo R$ 1999.9 
Nome: Notebook Asus R$ 3999.9 
Nota: 14629.3 
Espaço usado: 2.8164 


In [105]:
individuo1.mutacao(0.05)
individuo2.mutacao(0.05)

Antes ['0', '0', '1', '0', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1']
Depois ['0', '0', '1', '1', '1', '0', '0', '1', '1', '1', '0', '1', '1', '1']
Antes ['1', '0', '0', '1', '1', '0', '1', '0', '1', '0', '0', '0', '1', '1']
Depois ['1', '0', '0', '1', '1', '1', '1', '0', '1', '0', '0', '0', '1', '1']
